# Atualização incremental da tabela SCT type 2 'dim_funcionarios'  

## Inserção de dados fícticos

> [!IMPORTANT]  
> Sempre simule com dados após a última data já inserida

In [ ]:
%%sql

-- Insere novos valores de exemplo
INSERT INTO tb_log_eventos (Matricula, Nome, Filial, Departamento, Cargo, DataAdmissao, DataDesligamento, Operacao, DataLog) VALUES 
(98, 'FLAVIO FAGUNDES', 'CAMPINAS', 'PRODUCAO', 'SUPERVISOR DE PRODUCAO', '2021-12-07', '2025-01-28', 'DESLIGAMENTO', '2025-01-28 10:00:00'),
(143, 'JOSE DA SILVA', 'JUNDIAI', 'MANUTENCAO', 'ELETRICISTA DE MANUTENCAO XP', '2022-02-05', NULL, 'ALTERACAO', '2025-01-29 09:00:00');


## Criação da tabela 'staging_ultima_carga'  
> Esta tabela traz as últimas 'DataLog' e 'MatriculaSk'

In [ ]:
%%sql

-- Maiores 'DataLog' e 'MatriculaSk' na tabela 'dim_funcionarios'
DROP TABLE IF EXISTS staging_ultima_carga;
CREATE TABLE staging_ultima_carga AS
SELECT 
    MAX(DataLog) AS UltimaDataLog, 
    MAX(MatriculaSk) AS UltimaMatriculaSk 
FROM dim_funcionarios;

## Criação da 'staging_logs'  
> Tabela temporária com o último 'DataLog' e operações diferentes de 'DESLIGAMENTO'  

In [ ]:
%%sql
-- Tabela temporária com o log após o último 'DataLog' e quando 'Operacao' <> 'DESLIGAMENTO'
DROP TABLE IF EXISTS staging_logs;
CREATE TABLE staging_logs AS 
SELECT 
    Matricula,
    Nome, 
    Filial,
    Departamento,
    Cargo,
    DataAdmissao,
    DataDesligamento,
    DataLog,
    DataLog AS DataVigenciaInicial,

    -- Window function para trazer o próximo 'DataLog' por 'Matricula'
    LEAD(DataLog) OVER (PARTITION BY Matricula ORDER BY DataLog) AS ProximoDataLog,
    
    -- Window function para criar a coluna 'MatriculaSk' a partir da ordem do 'DataLog'
    -- partindo do maior já existente
    (SELECT COALESCE(UltimaMatriculaSk, 0) FROM staging_ultima_carga) + 
    ROW_NUMBER() OVER (ORDER BY DataLog) AS MatriculaSk  

FROM tb_log_eventos
WHERE DataLog > (SELECT UltimaDataLog FROM staging_ultima_carga)
    AND Operacao <> 'DESLIGAMENTO';

## Criação de 'staging_novos_eventos'  
> Tabela temporária contendo os novos eventos já tratados com vigências e 'MatriculaSk' prontos para insert.  

In [ ]:
%%sql

-- Tabela temporária contendo os novos eventos
DROP TABLE IF EXISTS staging_novos_eventos;
CREATE TABLE staging_novos_eventos AS
SELECT 
    MatriculaSk,  
    Matricula,
    Nome, 
    Filial,
    Departamento,
    Cargo,
    DataAdmissao,
    DataDesligamento,
    DataLog,

    -- 'DataVigenciaInicial' é a data de alteração
    CAST(DataVigenciaInicial AS DATE) AS DataVigenciaInicial,

    -- Coluna 'DataVigenciaFinal'
    -- Quando o 'ProximoDataLog' não estiver vazio, pega o
    -- 'ProximoDataLog' e subtrai um dia
    -- Caso estiver vazio então quer dizer que é o último
    -- Logo adiciona '9999-12-31'
    CASE 
        WHEN ProximoDataLog IS NOT NULL 
        THEN CAST(DATEADD(DAY, -1, ProximoDataLog) AS DATE)
        ELSE CAST('9999-12-31' AS DATE)
    END AS DataVigenciaFinal,

    -- 'EstaAtivo' traz 1 quando for o último evento
    CASE 
        WHEN ProximoDataLog IS NULL THEN 1 
        ELSE 0 
    END AS EstaAtivo,
    
    -- Criado o número da linha para capturar a menor 'DataLog' por Matricula
    ROW_NUMBER() OVER (PARTITION BY Matricula ORDER BY DataLog ASC) AS RN 

FROM staging_logs;

## Criação da 'staging_desligamentos'  
> Criação de tabela temporária com os desligamentos após a última carga.  

In [ ]:
%%sql

-- Tabela temporária com os desligamentos após a última carga
DROP TABLE IF EXISTS staging_desligamentos;
CREATE TABLE staging_desligamentos AS
SELECT 
    Matricula,
    Nome, 
    Filial,
    Departamento,
    Cargo,
    DataAdmissao,
    DataDesligamento,
    DataLog,
    CAST(DataLog AS DATE) AS DataVigenciaInicial
FROM tb_log_eventos
WHERE DataLog > (SELECT UltimaDataLog FROM staging_ultima_carga)
    AND Operacao = 'DESLIGAMENTO';


## Atualização da 'dim_funcionarios'  
> Atualiza a tabela 'dim_funcionarios' das linhas que estão ativas e tiveram atualização.   

In [ ]:
%%sql

-- Atualiza a tabela 'dim_funcionarios'
MERGE INTO dim_funcionarios AS f

-- Filtra apenas a menor 'DataLog' de cada 'Matricula' de 'staging_novos_eventos'
USING (SELECT * FROM staging_novos_eventos WHERE RN = 1) AS n  

-- nas linhas onde as matriculas correspondem e estão ativas
ON f.Matricula = n.Matricula AND f.EstaAtivo = 1

-- Atualizando os registros que já existem
WHEN MATCHED THEN 
    UPDATE SET 

        -- Atribuindo a data anterior a 'DataVigenciaInicial' de 'staging_novos_eventos'
        -- em 'DataVigenciaFinal' da tabela 'dim_funcionarios'
        f.DataVigenciaFinal = CAST(DATEADD(DAY, -1, n.DataVigenciaInicial) AS DATE),
        
        -- E coloca 'EstaAtivo' como 0
        f.EstaAtivo = 0;

## Inserção dos novos eventos
> Insere os novos eventos devidamente tratados na tabela 'dim_funcionarios'.  

In [ ]:
%%sql

-- Insere na tabela 'dim_funcionarios' todas as novas linhas em 'staging_novos_eventos'
INSERT INTO dim_funcionarios(
    MatriculaSk, 
    Matricula, 
    Nome, 
    Filial, 
    Departamento, 
    Cargo, 
    DataAdmissao, 
    DataDesligamento, 
    DataLog, 
    DataVigenciaInicial, 
    DataVigenciaFinal, 
    EstaAtivo
)
SELECT 
    MatriculaSk, 
    Matricula, 
    Nome, 
    Filial, 
    Departamento, 
    Cargo, 
    DataAdmissao, 
    DataDesligamento, 
    DataLog, 
    DataVigenciaInicial, 
    DataVigenciaFinal, 
    EstaAtivo
FROM staging_novos_eventos;

## Atualiza os desligamentos
> Realiza a atualização de desligamentos caso houver.  

In [ ]:
%%sql

-- Atualiza na tabela 'dim_funcionarios'
MERGE INTO dim_funcionarios AS f

-- Com os dados de 'staging_desligamentos'
USING (SELECT * FROM staging_desligamentos) AS d  

-- nas linhas onde as matriculas correspondem e estão ativas
ON f.Matricula = d.Matricula AND f.EstaAtivo = 1

-- Atualizando os registros que já existem
WHEN MATCHED THEN 
    UPDATE SET 

        -- As colunas 'DataDesligamento' e 'DataVigenciaFinal' em dim_funcionarios
        -- recebem o dia anterior da 'DataVigenciaInicial' da 'staging_desligamentos'
        f.DataDesligamento = CAST(DATEADD(DAY, -1, d.DataVigenciaInicial) AS DATE),
        f.DataVigenciaFinal = CAST(DATEADD(DAY, -1, d.DataVigenciaInicial) AS DATE),

        -- E coloca 'EstaAtivo' como 0
        f.EstaAtivo = 0;



## Exclue as tabelas temporárias  

In [ ]:
%%sql

-- Exclui as tabelas temporárias
DROP TABLE IF EXISTS staging_ultima_carga;
DROP TABLE IF EXISTS staging_logs;
DROP TABLE IF EXISTS staging_novos_eventos;
DROP TABLE IF EXISTS staging_desligamentos;
